In [1]:
from test_framework.messages import *
from test_framework.p2p import *
import socket
import binascii
import socket
import struct
import codecs
import time
import random
import hashlib
import binascii
from datetime import datetime
import io

#### Custom methods from https://github.com/bitcoin/bitcoin/tree/master/test/functional/test_framework

#### Settings

In [2]:
default_port    = 18333
magic_          = 0x0709110B
max_n_bits      = 0x1d00ffff
version_        = 70015
command         = b"version"
#host            = "8.9.3.218"
host            = "82.197.160.8"   # node Thush
MAGIC_BYTES     = b"\x0b\x11\x09\x07"

In [3]:
class Message:
    __slots__ = ("addrFrom", "nPortFrom", "addrTo", "nPortTo", "nNonce", "relay", "nServices",
                 "nStartingHeight", "nTime", "nVersion", "strSubVer","bMagic", "bCommand","timestamp")
    
    def __init__(self):
        self.nNonce = random.getrandbits(64)
        self.nStartingHeight = 596306
        self.relay = False
        self.nPortFrom = 0
        self.addrFrom = ""

    def serialize(self, command):
        self.bCommand = command
        # header 
        header     = b""
        header     += struct.pack("i",self.bMagic)                    # magic value                           
        header     += struct.pack("<12s",command)                     # command
        
        # payload
        payload    = b""
        payload    += struct.pack("i", self.nVersion)
        payload    += struct.pack("Q", self.nServices)
        payload    += struct.pack("q", int(time.time()))
        payload    += struct.pack("Q",0);
        
        payload    +=b"\x00" * 10 + b"\xff" * 2
        payload    += socket.inet_aton(self.addrTo)
        payload    += struct.pack(">H",self.nPortTo)                    # Receiver Port
        payload    += struct.pack("Q",0)
        payload    += struct.pack(">16s",bytes(self.addrFrom,'utf-8'))  # Sender IP Address
        payload    += struct.pack(">H",self.nPortFrom)                  # Sender Port
        payload    += struct.pack("<Q", self.nNonce)
        payload    += struct.pack("B",0)                                # Bytes in version string
        payload    += struct.pack("i", self.nStartingHeight)
        payload    += struct.pack("?", self.relay)
        
        # length and checksum
        length     = struct.pack("I", len(payload));
        checksum   = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[:4];
        
        return header + length + checksum + payload
    
    def deserialize(self, reply):
        self.bMagic          = struct.unpack("i",reply.read(4))[0]
        self.bCommand        = struct.unpack("<12s",reply.read(12))[0].split(b"\x00", 1)[0]
        length               = struct.unpack("I", reply.read(4))[0]
        checksum             = struct.unpack("I", reply.read(4))[0]
        self.nVersion        = struct.unpack("i", reply.read(4))[0]
        self.nServices       = struct.unpack("Q", reply.read(8))[0]
        self.timestamp       = datetime.fromtimestamp(struct.unpack("q", reply.read(8))[0])
        toServices           = struct.unpack("Q", reply.read(8))[0]
        # Skip 12 Bytes for IPv4
        reply.read(12)
        self.addrTo          = socket.inet_ntoa(reply.read(4))
        self.nPortTo         = struct.unpack(">H", reply.read(2))[0]
        fromServices         = struct.unpack("Q", reply.read(8))[0]
        # Skip 12 Bytes for IPv4
        reply.read(12)
        self.addrFrom        = socket.inet_ntoa(reply.read(4))
        self.nPortFrom       = struct.unpack(">H", reply.read(2))[0]
        self.nNonce          = struct.unpack("<Q", reply.read(8))[0]
        user_agent_byes      = struct.unpack("B", reply.read(1))[0]
        self.nStartingHeight = struct.unpack("i",reply.read(4))
        relay                = struct.unpack("?",reply.read(1))
        


In [4]:
message            = Message()
message.nVersion   = 70015
message.nServices  = 0
message.addrTo     = "82.197.160.8"
message.nPortTo    = 18333
message.bMagic     = magic_

In [5]:
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM);

reply_1 = Message()
reply_2 = Message()

sock.connect((message.addrTo,message.nPortTo));

sock.send(message.serialize(b'version'));
print(f"Sending {message.bCommand}")
time.sleep(1);
reply1 = sock.recv(1024);
reply_1.deserialize(io.BytesIO(reply1))

print(f"Received {reply_1.bCommand}")

sock.send(message.serialize(b'verack'));
print(f"Sending {message.bCommand}")
time.sleep(1);
reply2 = sock.recv(1024);
reply_2.deserialize(io.BytesIO(reply2))
print(f"Received {reply_2.bCommand}")

sock.close()

Sending b'version'
Received b'version'
Sending b'verack'
Received b'sendheaders'
